In [ ]:
import torch
from torch import nn

In [ ]:
class SmallCNN(nn.Module):
    def __init__(self):
        super(SmallCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 8, 3, padding=1)
        self.conv2 = nn.Conv2d(8, 16, 3, padding=1)  # 16 filters
        self.pool = nn.MaxPool2d(2,2)
        self.fc1 = nn.Linear(16*56*56, 3)

    def forward(self, x):
        x1 = self.pool(nn.ReLU()(self.conv1(x)))  # Save conv1 output
        x2 = self.pool(nn.ReLU()(self.conv2(x1))) # Save conv2 output
        x_flat = x2.view(-1, 16*56*56)
        out = self.fc1(x_flat)
        return out, x1, x2

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
from PIL import Image
import os
import random

img_folder = "/Users/benjaminbrooke/.cache/kagglehub/datasets/aryashah2k/breast-ultrasound-images-dataset/versions/1/Dataset_BUSI_with_GT/Train_data/benign"

random_choice = random.choice(os.listdir(img_folder))

img =  os.path.join(img_folder,random_choice)

img = Image.open(img)

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import transforms, datasets

Train_dataset = transforms.Compose([transforms.Resize((224, 224)), transforms.ToTensor()])
Test_dataset = transforms.Compose([transforms.Resize((224, 224)), transforms.ToTensor()])

train_dataset = datasets.ImageFolder(
    "/Users/benjaminbrooke/.cache/kagglehub/datasets/aryashah2k/breast-ultrasound-images-dataset/versions/1/Dataset_BUSI_with_GT/Train_data",
    Train_dataset)
test_dataset = datasets.ImageFolder(
    "/Users/benjaminbrooke/.cache/kagglehub/datasets/aryashah2k/breast-ultrasound-images-dataset/versions/1/Dataset_BUSI_with_GT/Test_data",
    Test_dataset)

train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=True)

In [ ]:
train_loader.batch_size

In [ ]:
train_dataset.classes

In [ ]:
def plot_kernels(layer, ncols=4):
    kernels = layer.weight.data.clone()
    n_filters = kernels.shape[0]

    nrows = (n_filters + ncols - 1) // ncols
    plt.figure(figsize=(ncols*2, nrows*2))

    for i in range(n_filters):
        k = kernels[i]
        # Normalize to 0-1
        k = (k - k.min()) / (k.max() - k.min())
        # Convert CxHxW -> HxWxC for plotting
        k_img = k.permute(1,2,0)
        plt.subplot(nrows, ncols, i+1)
        plt.imshow(k_img)
        plt.axis('off')
        plt.title(f"Kernel {i}")
    plt.show()


In [ ]:
def plot_image(img):
    img = img / 2 + 0.5  # unnormalize from [-1,1] to [0,1]
    npimg = img.numpy()
    # C x H x W → H x W x C
    npimg = np.transpose(npimg, (1, 2, 0))
    plt.imshow(npimg)
    plt.axis('off')
    plt.show()

In [ ]:
def plot_feature_maps(feature_map, ncols=4):
    # feature_map shape: (batch_size, channels, H, W)
    fmap = feature_map[0]  # first image in batch
    n_maps = fmap.shape[0]

    nrows = (n_maps + ncols - 1) // ncols
    plt.figure(figsize=(ncols*2, nrows*2))

    for i in range(n_maps):
        m = fmap[i]
        # Normalize to 0-1
        m = (m - m.min()) / (m.max() - m.min())
        plt.subplot(nrows, ncols, i+1)
        plt.imshow(m.detach().numpy(), cmap='gray')
        plt.axis('off')
        plt.title(f"Map {i}")
    plt.show()


In [ ]:
model = SmallCNN()



model.parameters

Absolutely — I'll walk you **step-by-step** through how a single image with shape:

```
[1, 3, 244, 244]
```

flows through your model, including **every convolution**, **ReLU**, **pooling**, **tensor sizes**, and **the exact final flatten size**.

---

# 🔍 **Your model**

```python
class SmallCNN(nn.Module):
    def __init__(self):
        super(SmallCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 8, 3, padding=1)
        self.conv2 = nn.Conv2d(8, 16, 3, padding=1)
        self.pool = nn.MaxPool2d(2,2)
        self.fc1 = nn.Linear(16*56*56, 3)

    def forward(self, x):
        x1 = self.pool(nn.ReLU()(self.conv1(x)))
        x2 = self.pool(nn.ReLU()(self.conv2(x1)))
        x_flat = x2.view(-1, 16*56*56)
        out = self.fc1(x_flat)
        return out, x1, x2
```

---

# 📌 Start with input

You feed **one image**:

```
x = [1, 3, 244, 244]
```

(batch size = 1)

---

# 🧮 **1. First Convolution Layer**

```
conv1 = nn.Conv2d(3, 8, kernel_size=3, padding=1)
```

### Formula:

Conv2d output size:

$$
[
H_\text{out} = \frac{H + 2P - K}{S} + 1
]
$$

* H = 244
* K = 3
* P = 1
* S = 1

$$
[
H_\text{out} = \frac{244 + 2(1) - 3}{1} + 1 = 244
]
$$

### ✔ Output after conv1:

```
[1, 8, 244, 244]
```

8 feature maps, each same size as input.

---

# ✴ **2. ReLU**

Shape stays the same:

```
[1, 8, 244, 244]
```

---

# 🟦 **3. MaxPool2d(2,2)**

Max pooling halves height and width:

$$
[
244 \rightarrow 122
]
$$

### ✔ Output after pool:

```
x1 = [1, 8, 122, 122]
```

This is **your first feature map output**.

---

# 🧮 **4. Second Convolution Layer**

```
conv2 = nn.Conv2d(8, 16, 3, padding=1)
```

Same formula:

$$
[
H_\text{out} = \frac{122 + 2(1) - 3}{1} + 1 = 122
]
$$

### ✔ Output after conv2:

```
[1, 16, 122, 122]
```

---

# ✴ **5. ReLU**

Shape stays:

```
[1, 16, 122, 122]
```

---

# 🟦 **6. MaxPool2d(2,2)**

$$
[
122 \rightarrow 61
]
$$

Pooling halves odd numbers by flooring.

### ✔ Output after pool:

```
x2 = [1, 16, 61, 61]
```

This is **your second feature map output**.

---

# 🔍 **7. Flatten**

You wrote:

```python
x_flat = x2.view(-1, 16*56*56)
```

But your actual feature map is:

```
[1, 16, 61, 61]
```

Total elements per sample:

$$
[
16 \times 61 \times 61 = 59536
]
$$

Since you force it to reshape to `16*56*56 = 50176`:

* 59536 ÷ 50176 ≈ 1.19 → ⚠ not divisible
* PyTorch instead produces a **wrong batch dimension**.

---

# 📏 **8. Final output: correct size**

After fixing:

```
out = [1, 3]
```

One sample → 3 output class logits.

---

# 📊 **Summary Table**

| Step | Layer   | Input → Output Size |
| ---- | ------- | ------------------- |
| 0    | Input   | `[1, 3, 244, 244]`  |
| 1    | Conv1   | `[1, 8, 244, 244]`  |
| 2    | ReLU    | `[1, 8, 244, 244]`  |
| 3    | Pool    | `[1, 8, 122, 122]`  |
| 4    | Conv2   | `[1, 16, 122, 122]` |
| 5    | ReLU    | `[1, 16, 122, 122]` |
| 6    | Pool    | `[1, 16, 61, 61]`   |
| 7    | Flatten | `[1, 59536]`        |
| 8    | FC1     | `[1, 3]`            |



In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 100  # Keep small for testing

for epoch in range(num_epochs):
    for batch_idx, (inputs, labels) in enumerate(train_loader):
        inputs, labels = inputs, labels

        optimizer.zero_grad()
        outputs, fmap1, fmap2 = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # Visualize kernels and feature maps after first batch of each epoch
        if batch_idx % 10 == 0:
            print("Label:", labels[0].item())
            plot_image(inputs[0])

            print(f"Epoch {epoch+1} Kernels conv1:")
            plot_kernels(model.conv1)

            print(f"Epoch {epoch+1} Feature maps conv1:")
            plot_feature_maps(fmap1)

            print(f"Epoch {epoch+1} Feature maps conv2:")
            plot_feature_maps(fmap2)

    print(f"Epoch [{epoch+1}/{num_epochs}] Loss: {loss.item():.4f}")
